In [ ]:
import sys
import pandas as pd
sys.path.append("..") # Adds higher directory to python modules path.
from MovieAnalysis import MovieAnalysis
import ast

MovieAnalysis()

In [ ]:
analysis = MovieAnalysis()
characters = analysis.character_data

In [ ]:
characters['Actor date of birth'].unique()

In [ ]:
characters['Actor date of birth'].info()

In [ ]:
characters['Actor date of birth'].isnull().sum()/len(characters['Actor date of birth'])

In [ ]:
characters.dropna(subset=['Actor date of birth'], inplace=True)

In [ ]:

import re

# Find unique formats using regex patterns
def detect_date_format(date_str):
    patterns = [
        (r'\d{4}-\d{2}-\d{2}', '%Y-%m-%d'),  # 2024-03-15
        (r'\d{2}-\d{2}-\d{4}', '%d-%m-%Y'),  # 15-03-2024
        (r'\d{2}/\d{2}/\d{4}', '%m/%d/%Y'),  # 03/15/2024
        (r'[A-Za-z]+ \d{1,2}, \d{4}', '%B %d, %Y'),  # March 15, 2024
        (r'\d{4}/\d{2}/\d{2}', '%Y/%m/%d'),  # 2024/03/15
        (r'^\d{4}-\d{2}$', '%Y-%m'),  # 2024-03
        (r'^\d{4}$', '%Y'),  # 2024
    ]
    
    for pattern, fmt in patterns:
        if re.match(pattern, date_str):
            return fmt
    return 'Unknown Format'

# Apply detection
characters['date_format'] = characters['Actor date of birth'].astype('str').apply(detect_date_format)

# Show unique formats
unique_formats = characters['date_format'].unique()

characters['date_format'].value_counts(normalize=True)

In [ ]:
characters[characters['date_format'] == 'Unknown Format']['Actor date of birth'].unique()

In [ ]:
characters['Actor date of birth'] = characters.apply(
    lambda row: pd.to_datetime(row['Actor date of birth'], format=row['date_format'], errors='coerce'),
    axis=1
)

In [ ]:
characters.sort_values('Actor date of birth', ascending=False).head()

In [ ]:
movies = analysis.movie_data
movies

In [ ]:
movies.dtypes

In [ ]:
print(type(movies['Movie genres'].iloc[0]))

In [ ]:
import pandas as pd
import ast

# Function to safely convert strings to dictionaries
def safe_eval(val):
    try:
        return ast.literal_eval(val)  # Convert only if it's a valid string representation of a dictionary
    except (ValueError, SyntaxError):
        return None  # Return None if conversion fails

# Apply function to the column
movies['Movie genres'] = movies['Movie genres'].str.strip().apply(safe_eval)

# Verify conversion
print(type(movies['Movie genres'].iloc[0]))  # Should print <class 'dict'>

# Convert dictionaries to lists of values (genre names)
movies['Movie genres'] = movies['Movie genres'].apply(lambda x: list(x.values()) if isinstance(x, dict) else [])

# Explode to separate rows
movies = movies.explode('Movie genres')


movies


In [ ]:
movies['Movie genres'].value_counts().head(10)